## <center> Практическое задание к лекции 1.

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

1. Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

- Expected reciprocal rank (ERR)  
- PFound 
- F1 score

2. Доделать все функции, где стоит комментарий "сделать дома"

In [1]:
import numpy as np

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров

# user1
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [6]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    # сделать в домашней работе
    return hit_rate(recommended_list[:k], bought_list)

In [4]:
hit_rate(recommended_list, bought_list)

1

In [7]:
hit_rate_at_k(recommended_list, bought_list)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

In [10]:
prices_recommended=[400,40,30,120,99,144,25,90,90,320]

In [11]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [9]:
precision(recommended_list, bought_list)

0.2

In [12]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [13]:
money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5)

0.7547169811320755

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [29]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    #сделать дома
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # сделать дома
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought=np.array(prices_bought)
    
    flags = np.isin(recommend_list, bought_list)
    
    recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    
    return recall
    

In [23]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
prices_recommended=[400, 40, 30, 120, 99, 144, 25, 90, 90, 320]
# user1
bought_list = [521, 32, 143, 991]
prices_bought=[60, 40, 400, 120]

In [24]:
recall(recommended_list, bought_list)

0.5

In [25]:
recall_at_k(recommended_list, bought_list)

0.5

In [30]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.8387096774193549

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [33]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    for index_relevant in relevant_indexes:
        sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    
    return sum_/amount_relevant

In [34]:
ap_k(recommended_list, bought_list, k=5)

0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [61]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]
                      ] # юзер 3

In [44]:
def map_k(recommended_list, bought_list, k=5):
    # сделать дома
    sum_apk = 0
    for rec, bought in zip(recommended_list, bought_list):
        sum_apk += ap_k(rec, bought, k)
    result = sum_apk/len(bought_list)
    return result

In [49]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [50]:
# по желанию
def dcg(list_of_indexes, lenght):
    result = 0
    for i in list_of_indexes:
        if i == 0:
            result += 1/1
        elif i == 1:
            result += 1/2
        else:
            result += 1/np.log(i+1)
    return result/lenght
    

def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommend_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    ndcg = dcg(relevant_indexes, k)/dcg([index for index in range(k)], k)

    return ndcg

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [54]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    # сделать дома
    bought_list = np.array(bought_list)
    recommend_list = np.array(recommended_list)[:k]
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    reciprocal_rank = 1 / (relevant_indexes[0] + 1)
    return reciprocal_rank


def mean_reciprocal_rank(recommended_lists, bought_lists, k=1):
    # сделать дома
    reciprocal_rank_list = []
    for rec, bougth in range(len(recommended_lists)):
        bought_list = np.array(bought_lists[i])
        recommend_list = np.array(recommended_lists[i])[:k]
        reciprocal_rank_list.append(reciprocal_rank(recommend_list, bought_list, k))
    return np.mean(reciprocal_rank_list)

In [59]:
reciprocal_rank(recommended_list, bought_list, k=5)

1.0

In [63]:
mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333